# 감성분석
- 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법으로 소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용
- 주관적인 단어와 문맥을 기반으로 감성(Sentiment) 수치를 계산
- 감성지수 -> 긍정 감성 지수, 부정 감성 지수
- 이들 지수를 합산하여 긍정 감성 또는 부정 감성을 결정

## 1. 지도 학습 기반

- 지도학습을 이용한 감성분석
    - 학습 데이터, 타겟 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤, 이를 기반으로 다른 데이터의 감성 분석을 예측    

## 영화평 감성분석

#### 데이터 불러오기

In [11]:
import pandas as pd
review_df = pd.read_csv("./labeledTrainData.tsv/labeledTrainData.tsv",
                       header = 0, sep="\t", quoting=3)

In [3]:
review_df
# sentiment : 영화평 (1:긍정, 0:부정)
# id : 각 데이터의 id
# review : 영화평의 텍스트

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
...,...,...,...
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."


In [4]:
print(review_df["review"][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

#### 정규표현식을 이용한 텍스트 전처리 

In [12]:
import re
# <br> html 태그는 replace 함수로 공백으로 변환
review_df["review"] = review_df["review"].str.replace("<br />"," ")

# 파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
# re.sub(바꿀정규표현식, 이걸로 바꿀 문자열, 바꿀 문자열)
# [] 밖에서 ^ 사용할 경우 맨 앞부터 만족해야 한다는 뜻
# [] 안에서 ^ 사용할 경우 []안에 들어간 것의 반대. [] 안에 들어간 것 외에 것.
review_df["review"] = review_df["review"].apply(lambda x: re.sub("[^a-zA-Z]"," ",x))

In [13]:
review_df["review"][0]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

### 피쳐 추출 및 훈련, 예측

In [16]:
from sklearn.model_selection import train_test_split

class_df = review_df["sentiment"]
feature_df = review_df.drop(["id","sentiment"], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df,
                                                   test_size=0.3, random_state=156)
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

#### countvectorizer 기반

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 english -> CountVectorization 수행
pipeline = Pipeline([
    ("cnt_vect", CountVectorizer(stop_words="english")),
    ("lr_clf", LogisticRegression(C=10))
])

# Pipeline 객체를 이용해 fit, predict
# predict_proba()는 roc_auc 때문에 수행 -> roc_auc는 임계값을 변화시키면서 수행
pipeline.fit(X_train["review"], y_train)
pred = pipeline.predict(X_test["review"])
pred_probs = pipeline.predict_proba(X_test["review"])[:,1]

print("예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}".format(accuracy_score(y_test, pred),
                                                roc_auc_score(y_test, pred_probs)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


예측 정확도는 0.8685, ROC-AUC는 0.9372


#### tfidf 기반

In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 english -> CountVectorization 수행
pipeline = Pipeline([
    ("cnt_vect", TfidfVectorizer(stop_words="english")),
    ("lr_clf", LogisticRegression(C=10))
])

# Pipeline 객체를 이용해 fit, predict
# predict_proba()는 roc_auc 때문에 수행 -> roc_auc는 임계값을 변화시키면서 수행
pipeline.fit(X_train["review"], y_train)
pred = pipeline.predict(X_test["review"])
pred_probs = pipeline.predict_proba(X_test["review"])[:,1]

print("예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}".format(accuracy_score(y_test, pred),
                                                roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8916, ROC-AUC는 0.9568


## 2. 비지도학습 기반
- "Lexicon"이라는 감성 어휘 사전을 이용하여 긍정적, 부정적 감성 여부를 판단
    - Lexicon은 각 단어의 긍정/부정 감성의 정도를 의미하는 수치(감성 지수)를 가지고 있음
    - 감성지수는 단어의 위치, 주변 단어, 문맥, POS(part of speech)등을 참고해 결정

#### 대표적 패키지

-  nlp 패키지의 wordnet
    - 단어를synset이라는 개념을 이용해 표현 
    - 어휘의 시맨틱 정보를 제공(각각의 품사, 문맥 등을 제공)   
*시맨틱(semantic) : 문맥상 의미(동일한 단어나 문장일지라도 다른 환경과 문맥에서는 다르게 표현, 이해)*

- SentiWordNet : nltk 패키지의 wordnet과 유사하게 감성 단어 전용의 wordnet을 구현, wordnet의 synset 개념을 감성 분석에 적용 -> 예측성능이 좋지 않음
    - 감성지수(긍정/부정), 객관성 지수 존재 => 세개를 합하면 1이 되어야 함.
        - 감성지수
            - 긍정 감성지수 : 해당 단어가 감성적으로 얼마나 긍정적인지
            - 부정 감성지수 : 해당 단어가 감성적으로 얼마나 부정적인지
        - 객관성 지수 : 감성과 관계없이 얼마나 객관적인지를 수치로 나타냄
            - 객관성 지수가 1으로 완전 객관적 단어이면 감성지수가 존재하지 않음
    - 문장별로 단어들의 긍정 감성 지수와 부정 감성 지수를 합산하여 최종 감성 지수를 계산하고 이에 기반해 감성이 긍정인지 부정인지를 결정
- VADER : 주로 소셜 미디어의 텍스트에 대한 감성분석을 제공하기 위한 패키지, 뛰어난 감성 분석 결과를 제공하며, 비교적 빠른 수행 시간을 보장해 대용량 텍스트 데이터에 잘 사용됨
- Pattern : 예측 성능면에서 가장 주목받음. 파이썬2에서만 동작

### SentiWordNet을 이용한 감성 분석

In [32]:
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown.zip.
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |  

[nltk_data]    |   Unzipping corpora\qc.zip.
[nltk_data]    | Downloading package reuters to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package rte to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\rte.zip.
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\senseval.zip.
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\sentiwordnet.zip.
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\sentence_polarity.zip.
[nltk_data]    | Downloading package shakespeare to
[nltk_data]   

[nltk_data]    | Downloading package porter_test to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping stemmers\porter_test.zip.
[nltk_data]    | Downloading package wmt15_eval to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping models\wmt15_eval.zip.
[nltk_data]    | Downloading package mwa_ppdb to
[nltk_data]    |     C:\Users\이혜림\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping misc\mwa_ppdb.zip.
[nltk_data]    | 
[nltk_data]  Done downloading collection all


True

In [22]:
from nltk.corpus import wordnet as wn

term = "present"

# "present" 라는 단어로 wordnet의 synsets 생성.
synsets = wn.synsets(term)
print("synsets()반환 type:", type(synsets))
print("synsets()반환 값 개수:", len(synsets))
print("synsets()반환 값 :", synsets)
# synset(단어.품사.의미index)

synsets()반환 type: <class 'list'>
synsets()반환 값 개수: 18
synsets()반환 값 : [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [25]:
# synset객체가 가지는 속성 : 품사, 정의, 부명제 등의 시맨틱적인 요소를 표현
for synset in synsets:
    print(synset.name())
    print(synset.lexname())
    print(synset.definition())
    print(synset.lemma_names())

present.n.01
noun.time
the period of time that is happening now; any continuous stretch of time including the moment of speech
['present', 'nowadays']
present.n.02
noun.possession
something presented as a gift
['present']
present.n.03
noun.communication
a verb tense that expresses actions or states at the time of speaking
['present', 'present_tense']
show.v.01
verb.perception
give an exhibition of to an interested audience
['show', 'demo', 'exhibit', 'present', 'demonstrate']
present.v.02
verb.communication
bring forward and present to the mind
['present', 'represent', 'lay_out']
stage.v.01
verb.creation
perform (a play), especially on a stage
['stage', 'present', 'represent']
present.v.04
verb.possession
hand over formally
['present', 'submit']
present.v.05
verb.stative
introduce
['present', 'pose']
award.v.01
verb.possession
give, especially as an honor or reward
['award', 'present']
give.v.08
verb.possession
give as a present; make a gift of
['give', 'gift', 'present']
deliver.v.01


In [26]:
# synset 객체를 단어별로 생성
tree = wn.synset("tree.n.01")
lion = wn.synset("lion.n.01")
tiger = wn.synset("tiger.n.02")
cat = wn.synset("cat.n.01")
dog = wn.synset("dog.n.01")

entities = [tree, lion, tiger, cat, dog]
similarities = []
# entity name은 .을 기준으로 단어.품사.index 로 구성되어 있기 때문
entity_names = [entity.name().split(".")[0] for entity in entities]

# 단어별 snset들을 반복하면서 다른 단어들의 synset과 유사도를 측정
for entity in entities:
    similarity = [round(entity.path_similarity(compared_entity),2) for compared_entity in entities]
    similarities.append(similarity)

In [27]:
similarities

[[1.0, 0.07, 0.07, 0.08, 0.12],
 [0.07, 1.0, 0.33, 0.25, 0.17],
 [0.07, 0.33, 1.0, 0.25, 0.17],
 [0.08, 0.25, 0.25, 1.0, 0.2],
 [0.12, 0.17, 0.17, 0.2, 1.0]]

In [33]:
import nltk
from nltk.corpus import sentiwordnet as swn
term = 'slow'

synsets = list(swn.senti_synsets(term))
print("synsets()반환 type:", type(synsets))
print("synsets()반환 값 개수:", len(synsets))
print("synsets()반환 값 :", synsets)

synsets()반환 type: <class 'list'>
synsets()반환 값 개수: 11
synsets()반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [37]:
# father -> 객관적단어, 객관적 지수 :1, 감성지수 존재하지 X
father = swn.senti_synset("father.n.01")
print(father.pos_score())
print(father.neg_score())
print(father.obj_score())

# famous -> 어느정도 객관적 단어, 어느정도 긍정적 단어
famous = swn.senti_synset("famous.a.01")
print(famous.pos_score())
print(famous.neg_score())
print(famous.obj_score())

0.0
0.0
1.0
0.375
0.0
0.625


### SentiWordnet을 이용한 영화 감상평 감성 분석
1. 문서를 문장 단위로 분해
1. 문장을 단어 단위로 분해, 품사 태깅
1. 태깅된 단어를 기반으로 synset 객체와 senti-synset 객체를 생성
1. senti-synset에서 긍정감성/부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정

In [70]:
# 간단한 nltk penntreebank tag를 기반으로 wordnet 기반의 품사 tag로 변환
def penn_to_wn(tag):
    # str.startwith("문자열") -> 해당 str이 문자열로 시작하는지에 따라 bool 형식 반환
    if tag.startswith("J"):
        return wn.ADJ
    elif tag.startswith("N"):
        return wn.NOUN
    elif tag.startswith("R"):
        return wn.ADV
    elif tag.startswith("V"):
        return wn.VERB

In [71]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화
    sentiment = 0.
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text) # 문장 토큰화
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출
        # pos_tag는 각 단어에 품사를 태깅해서 [(단어, 품사)]로 반환
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag) # 기본형으로 바꿈
            if not lemma:
                continue
            
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체 생성
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            synset = synsets[0] # 첫번째 단어일 확률이 가장 높기 때문에 index=0인 단어 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score()-swn_synset.neg_score())
            tokens_count += 1
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정, 그렇지 않을 경우 부정 반환
    if sentiment >=0:
        return 1
    
    return 0
    

In [72]:
# 각 행별 review에 감성분석을 시행하여 새로운 열로 추가해줌
review_df["preds"] = review_df["review"].apply(lambda x: swn_polarity(x))
y_target = review_df["sentiment"].values
preds = review_df["preds"].values

In [73]:
# 예측 성능 평가
from sklearn.metrics import accuracy_score , confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
print(confusion_matrix(y_target, preds))
print("정확도", accuracy_score(y_target, preds))
print("정밀도", precision_score(y_target, preds))
print("재현율", recall_score(y_target,preds))

[[7668 4832]
 [3636 8864]]
정확도 0.66128
정밀도 0.647196261682243
재현율 0.70912


### VADER을 이용한 감성분석
- VADER 만의 Lexicon(감성 사전)이 존재
- 소셜 미디어의 감성 분석 용도로 만들어졌음
- 전체 text를 바로 넣기만 하면 바로 감성분석이 실행됨.
    - nltk.sentiwordnet의 경우 전체 텍스트에 대해 문장 토큰화, 단어토큰화 실행, 품사 태깅, wordnet기반의 단어로 바꾸고 감성 분석을 실행

In [80]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df["review"][0])
print(senti_scores)
# 부정적인 감성 지수, 객관적인 감성지수, 긍정적인 감성지수
# compound -> 이를 적절히 조합하여 -1~1 사이의 감성 지수
# compound가 0.1 이상이면 긍정 감성, 이하이면 부정감성

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


In [84]:
def vader_polarity(review, threshold = 0.1): # compound 의 임계값
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # compound 값에 기반해 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환
    agg_score = scores["compound"]
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

In [85]:
# apply lambda 식을 이용해 레코드별로 vader_polarity()를 수행하고 결과를 "vader_preds"에 저장
review_df["varder_preds"] = review_df["review"].apply(lambda x:vader_polarity(x,0.1))
y_target = review_df["sentiment"].values
vader_preds = review_df["varder_preds"].values

print(confusion_matrix(y_target,vader_preds))
print("정확도", accuracy_score(y_target, vader_preds))
print("정밀도", precision_score(y_target, vader_preds))
print("재현율", recall_score(y_target,vader_preds))

[[ 6736  5764]
 [ 1867 10633]]
정확도 0.69476
정밀도 0.6484722815149113
재현율 0.85064


varder lexicon이 senti-wordnet lexicon보다 더 나은 재현율을 반환.